In [70]:
def egg_thermophysical_properties(T, water_content=None, product_type='whole_egg'):
    # Check for albumen solidification (typically occurs around 60-65°C = 333-338K)
    if T > 333:  # 60°C = 333K
        return {
            'valid': False,
            'message': f'Warning: Temperature {T}K ({T-273.15:.1f}°C) exceeds typical albumen solidification point (60°C). Properties may not be accurate.',
            'rho': None,
            'Cp': None,
            'k': None
        }
    
    # Default water contents based on product type (from Table 1)
    default_water_contents = {
        'yolk': 0.518,      # 51.8% water
        'whole_egg': 0.664,  # 66.4% water (approximately 40% white, 60% yolk blend)
        'white': 0.882,      # 88.2% water
        'blend': 0.736       # 73.6% water (approximately 60% white, 40% yolk blend)
    }
    
    # Use provided water_content or default based on product_type
    if water_content is None:
        if product_type in default_water_contents:
            W_w = default_water_contents[product_type]
        else:
            W_w = default_water_contents['whole_egg']  # fallback
    else:
        W_w = water_content
    
    # Validate water content range
    if W_w < 0.518 or W_w > 0.882:
        return {
            'valid': False,
            'message': f'Water content {W_w} outside valid range (0.518-0.882)',
            'rho': None,
            'Cp': None,
            'k': None
        }
    
    # Coefficients from Table 5 in the paper
    # Thermophysical property = a + b*T + c*W_w
    
    # Density (kg/m^3)
    rho = 1295.72 - 0.0559 * T - 284.43 * W_w
    
    # Heat capacity (J/gK)
    Cp = 0.668 + 0.0025 * T + 2.4429 * W_w
    
    # Thermal conductivity (W/mK)
    k = 0.276 - 0.0004 * T + 0.4302 * W_w
    
    # Validate temperature range (273-311K from the study)
    temp_valid = 273 <= T <= 311
    
    return {
        'valid': temp_valid,
        'message': 'Properties calculated successfully' if temp_valid else f'Warning: Temperature {T}K outside experimental range (273-311K)',
        'rho': rho,      # kg/m^3
        'Cp': Cp,        # J/gK
        'k': k,          # W/mK
        'temperature_K': T,
        'temperature_C': T - 273.15,
        'water_content': W_w,
        'product_type': product_type
    }

# Testing
if __name__ == "__main__":
    # Test at different temperatures
    test_temps = [280, 300, 320, 340]  # Kelvin as unit
    
    for temp in test_temps:
        result = egg_thermophysical_properties(temp)
        
        print(f"\nTemperature: {temp}K ({temp-273.15:.1f}°C)")
        print(f"Status: {result['message']}")
        
        if result['valid'] or result['rho'] is not None:
            print(f"Density (rho): {result['rho']:.2f} kg/m^3")
            print(f"Heat Capacity (Cp): {result['Cp']:.3f} J/gK")
            print(f"Thermal Conductivity (k): {result['k']:.3f} W/mK")
            print(f"Water Content: {result['water_content']:.3f}")
            print(f"Product Type: {result['product_type']}")
        print("-" * 30)
    
    # Test different product types
    print("\nDifferent Product Types at 300K:")
    for product in ['yolk', 'whole_egg', 'white']:
        result = egg_thermophysical_properties(300, product_type=product)
        print(f"{product:10} - rho: {result['rho']:.1f} kg/m^3, Cp: {result['Cp']:.2f} J/gK")


Temperature: 280K (6.9°C)
Status: Properties calculated successfully
Density (rho): 1091.21 kg/m^3
Heat Capacity (Cp): 2.990 J/gK
Thermal Conductivity (k): 0.450 W/mK
Water Content: 0.664
Product Type: whole_egg
------------------------------

Temperature: 300K (26.9°C)
Status: Properties calculated successfully
Density (rho): 1090.09 kg/m^3
Heat Capacity (Cp): 3.040 J/gK
Thermal Conductivity (k): 0.442 W/mK
Water Content: 0.664
Product Type: whole_egg
------------------------------

Temperature: 320K (46.9°C)
Status: Warning: Temperature 320K outside experimental range (273-311K)
Density (rho): 1088.97 kg/m^3
Heat Capacity (Cp): 3.090 J/gK
Thermal Conductivity (k): 0.434 W/mK
Water Content: 0.664
Product Type: whole_egg
------------------------------

Temperature: 340K (66.9°C)
Status: Warning: Temperature 340K (66.9°C) exceeds typical albumen solidification point (60°C). Properties may not be accurate.
------------------------------

Different Product Types at 300K:
yolk       - rho